In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e1/sample_submission.csv
/kaggle/input/playground-series-s4e1/train.csv
/kaggle/input/playground-series-s4e1/test.csv


In [2]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [3]:
# Load the datasets
train_data = pd.read_csv('/kaggle/input/playground-series-s4e1/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s4e1/test.csv')

In [4]:
# Separate features and target variable
X = train_data.drop(['Exited'], axis=1)
y = train_data['Exited']

In [5]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
# Define numerical and categorical features
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns


In [7]:
# Create preprocessing pipeline for XGBoost
numeric_transformer_xgb = StandardScaler()

preprocessor_xgb = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_xgb, numerical_features),
    ])
# Create preprocessing pipeline for CatBoost
categorical_transformer_cat = OneHotEncoder(handle_unknown='ignore')

In [8]:
preprocessor_cat = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_xgb, numerical_features),
        ('cat', categorical_transformer_cat, categorical_features)
    ])

In [9]:
# Append classifier to preprocessing pipeline
cat_clf = Pipeline(steps=[('preprocessor', preprocessor_cat),
                           ('classifier', CatBoostClassifier(iterations=100, random_state=42, verbose=0))])

In [10]:
# Fit the CatBoost model
cat_clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['id', 'CustomerId', 'CreditScore', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Surname', 'Geography', 'Gender'], dtype='object'))])),
                ('classifier',
                 <catboost.core.CatBoostClassifier object at 0x784c6bcbd390>)])

In [11]:
test_predictions_cat = cat_clf.predict_proba(test_data)[:, 1]

In [12]:
# Create submission file
submission_df = pd.DataFrame({'id': test_data['id'], 'Exited': test_predictions_cat})
submission_df.to_csv('submission.csv', index=False)